In [1]:
#当当网2020-2023年图书榜单数据清洗流程
#核心功能：
# 1.异常值处理 | 2.缺失值填充 | 3.数据去重 | 4.特征工程
#导入必要库
import pandas as pd
import numpy as np

In [2]:
### 1.数据加载与初检
df= pd.read_excel('C:\\数据分析\\data.xlsx')
df.head()  #前5行数据
df.info()  #查看数据整体信息
df.describe()  #查看数据描述

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   书名      2000 non-null   object        
 1   作者      1997 non-null   object        
 2   出版日期    1999 non-null   datetime64[ns]
 3   出版社     2000 non-null   object        
 4   原价      2000 non-null   float64       
 5   售价      2000 non-null   float64       
 6   折扣比例    2000 non-null   object        
 7   排序      2000 non-null   int64         
 8   排行榜类型   2000 non-null   datetime64[ns]
 9   推荐值     2000 non-null   float64       
 10  电子书价格   693 non-null    float64       
 11  评论数     1996 non-null   float64       
dtypes: datetime64[ns](2), float64(5), int64(1), object(4)
memory usage: 187.6+ KB


,出版日期,原价,售价,排序,排行榜类型,推荐值,电子书价格,评论数
count,1999,2000.000000,2000.000000,2000.000000,2000,2000.000000,693.000000,1.996000e+03
mean,2018-04-20 15:07:39.229614848,61.707300,32.415550,250.500000,2021-07-02 06:00:00.000000256,0.997150,25.323463,3.771247e+05
min,1982-03-01 00:00:00,14.800000,2.100000,1.000000,2020-01-01 00:00:00,0.000000,0.000000,4.900000e+01
25%,2017-04-01 00:00:00,29.900000,14.775000,125.750000,2020-10-01 12:00:00,0.999000,7.990000,1.283530e+05
50%,2018-12-01 00:00:00,45.000000,24.000000,250.500000,2021-07-02 12:00:00,1.000000,20.990000,2.395670e+05
75%,2020-07-01 00:00:00,64.000000,35.800000,375.250000,2022-04-02 06:00:00,1.000000,35.400000,4.268780e+05
max,2023-12-01 00:00:00,828.000000,455.400000,500.000000,2023-01-01 00:00:00,1.000000,405.000000,3.427401e+06
std,NaN,66.719261,34.713208,144.373376,NaN,0.044796,28.814382,4.221029e+05


In [3]:
### 2.数据清洗
# 2.1异常值修正
df.loc[df['推荐值'] == 0.0, '推荐值'] = 1.0  #修正无效推荐值

In [4]:
# 2.2缺失值处理
df.isnull().sum()  #检查各变量中缺失值的数量

书名          0
作者          3
出版日期        1
出版社         0
原价          0
售价          0
折扣比例        0
排序          0
排行榜类型       0
推荐值         0
电子书价格    1307
评论数         4
dtype: int64

In [5]:
#删除缺失率>90%的字段
df.drop(columns='电子书价格',inplace=True)

In [6]:
#关键字段手工补全 (通过ISBN查询)
df.loc[df.isnull().any(axis=1)]  #定位缺失值所在的行
df.loc[233,'作者'] = '全国一级建造师执业资格考试用书编写委员会'
df.loc[731,'作者'] = '中国共产党简史编写组'
df.loc[1908,'作者'] = '伊泽'
df.loc[233,'出版日期'] = '2020-05-01'  #处理作者字段的3个缺失值和1个出版日期缺失值

In [7]:
#数值字段智能填充
df['评论数'] = df['评论数'].fillna(df['评论数'].mean())  #处理评论数的4个缺失值，用平均数填充

In [8]:
# 2.3数据去重
df.drop_duplicates(inplace = True)

In [9]:
### 3.特征工程
# 3.1数据标准化
df[['原价','售价']] = df[['原价','售价']].astype(float).round(1)
df['折扣比例'] = df['折扣比例'].str.replace('折','').astype(float).round(1)
df['推荐值'] = df['推荐值'].astype(float).round(3)
df['评论数'] = df['评论数'].astype(int)
df['排行榜类型'] = pd.to_datetime(df['排行榜类型']).dt.year  #清理排行榜类型，只保留年份

In [10]:
# 3.2构建时间维度特征
#将排序、排行榜类型替换为2020排名、2021排名、2022排名、2023排名等四个字段
years = [2020, 2021, 2022, 2023]
for year in years:
    df[f'{year}排名'] = np.where(df['排行榜类型'] == year, df['排序'], np.nan)
df = df.drop(columns = ['排序','排行榜类型'])

In [11]:
# 3.3聚合与衍生特征
#用书名聚合(取每组第一个非空值)
df = df.groupby('书名').first()
df[['2020排名','2021排名','2022排名','2023排名']] = df[['2020排名','2021排名','2022排名','2023排名']].fillna(0).astype(int)  #填充NAN值，调整格式
#新增字段：上榜次数、平均排名
df['上榜次数'] = (df[['2020排名','2021排名','2022排名','2023排名']] !=0).sum(axis=1).astype(int)
df['平均排名'] = (df[['2020排名','2021排名','2022排名','2023排名']].sum(axis=1)/df['上榜次数']).astype(float).round(2)

In [12]:
### 4.结构验证与数据导出
df.info()
df.to_excel('C:\\数据分析\\data_cleaned.xlsx', index=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0-3岁专注力训练贴纸书（全6册）幼儿思维逻辑开发游戏书贴贴画邦 to 龙猫（宫崎骏代表作首次授权。吉卜力官方授权唯一简体中文版绘本
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   作者      1094 non-null   object        
 1   出版日期    1094 non-null   datetime64[ns]
 2   出版社     1094 non-null   object        
 3   原价      1094 non-null   float64       
 4   售价      1094 non-null   float64       
 5   折扣比例    1094 non-null   float64       
 6   推荐值     1094 non-null   float64       
 7   评论数     1094 non-null   int64         
 8   2020排名  1094 non-null   int64         
 9   2021排名  1094 non-null   int64         
 10  2022排名  1094 non-null   int64         
 11  2023排名  1094 non-null   int64         
 12  上榜次数    1094 non-null   int64         
 13  平均排名    1094 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(6), object(2)
memory usage: 128.2+ KB
